In [1]:
#import libraries
import pandas as pd
import numpy as np
import re

#import functions
from functions import get_sample_id
from functions import panda_stripper

## Extract and transform main table

In [2]:
#read excel table and strip
deer_table = pd.read_excel('data/Muledeer_2021_22_Sample sheet.xlsx', usecols=[0,1,2,3,4,5])
deer_table = panda_stripper(deer_table)

#rename columns
deer_table.columns = ['sample_id', 'collar_id', 'species', 'sex', 'capture_date', 'capture_unit']

In [3]:
deer_table.head()

,sample_id,collar_id,species,sex,capture_date,capture_unit
0,BC1101,50713,Mule Deer,Female,2021-12-11,Book Cliffs
1,BC1102,50716,Mule Deer,Female,2021-12-11,Book Cliffs
2,BC1103,50714,Mule Deer,Female,2021-12-11,Book Cliffs
3,BC1104,50720,Mule Deer,Female,2021-12-11,Book Cliffs
4,BC1105,50722,Mule Deer,Male,2021-12-11,Book Cliffs


## Extract and Transform Lab Results

### Adenovirus

In [4]:
#read and strip table
adenovirus_df = pd.read_excel('data/deer_tables.xlsx', sheet_name='adenovirus')
adenovirus_df = panda_stripper(adenovirus_df)

#get sample id
adenovirus_df = get_sample_id(adenovirus_df, 'sample_id')

#combine result1 and result2 cols
adenovirus_df['adenovirus_result'] = adenovirus_df.apply(lambda row: row['result1'] if row['result1'] is not np.NaN else row['result2'], axis=1)
del adenovirus_df['result1']
del adenovirus_df['result2']

#trim "Negative @" result
adenovirus_df['adenovirus_result'] = adenovirus_df['adenovirus_result'].apply(lambda row: row.split()[0])

#merge with deer_table
deer_table = deer_table.merge(adenovirus_df, on='sample_id', how='outer')

### ehdv

In [5]:
#read and strip table
ehdv_df = pd.read_excel('data/deer_tables.xlsx', sheet_name='ehdv', usecols=[0,2])
ehdv_df = panda_stripper(ehdv_df)

#get sample_id and drop original col
ehdv_df = get_sample_id(ehdv_df, 'Animals::Specimens')
del ehdv_df['Animals::Specimens']

#rename cols
ehdv_df.columns = ['ehdv_result', 'sample_id']

#merge with deer_table
deer_table = deer_table.merge(ehdv_df, on='sample_id', how='outer')


### bluetongue

In [6]:
#read and strip table
bluetongue_df = pd.read_excel('data/deer_tables.xlsx', sheet_name='bluetongue')
bluetongue_df = panda_stripper(bluetongue_df)

#get sample_id and drop original col
bluetongue_df = get_sample_id(bluetongue_df, 'specimen')
del bluetongue_df['specimen']

#rename cols
bluetongue_df.columns = ['bluetongue_result', 'sample_id']

#merge with deer_table
deer_table = deer_table.merge(bluetongue_df, on='sample_id', how='outer')

## Load

### Save to .xlsx file

In [7]:
deer_table.to_excel('data/finals/Mule Deer 2021-2022 Lab Results.xlsx', index=False)